In [1]:
!pip install fastapi uvicorn nest_asyncio pyngrok gradio



In [4]:

!pip install gradio


In [5]:
import gradio as gr
import random
from datetime import datetime, timedelta

# Major cities in Gujarat
gujarat_cities = ["Ahmedabad", "Surat", "Vadodara", "Rajkot", "Bhavnagar", "Junagadh", "Gandhinagar", "Jamnagar", "Porbandar", "Anand"]

# Major cities in India
india_cities = [
    "Mumbai", "Delhi", "Bengaluru", "Chennai", "Kolkata", "Hyderabad", "Pune", "Jaipur", "Lucknow", "Patna",
    "Chandigarh", "Bhopal", "Nagpur", "Indore", "Ranchi", "Goa", "Trivandrum", "Kochi", "Agra", "Varanasi"
]

all_cities = gujarat_cities + india_cities

# Dummy bus operator names
operators = ["GSRTC", "RedBus", "VRL Travels", "SRS Travels", "Neeta Travels", "Patel Travels"]

# Store bookings
bookings = []


In [6]:
def generate_dummy_buses(source, destination, travel_date):
    results = []
    for i in range(3):  # generate 3 buses per query
        bus = {
            "id": random.randint(100, 999),
            "source": source,
            "destination": destination,
            "time": (datetime.now() + timedelta(hours=random.randint(1, 12))).strftime("%H:%M"),
            "price": random.randint(400, 1500),
            "operator": random.choice(operators),
            "date": travel_date
        }
        results.append(bus)
    return results


def bus_booking_agent(message, history=[]):
    message = message.lower()

    if "to" in message and "on" in message:
        try:
            # Parse input
            parts = message.split(" on ")
            route = parts[0].split(" to ")
            source, destination = route[0].strip().title(), route[1].strip().title()
            travel_date = parts[1].strip()

            if source not in all_cities or destination not in all_cities:
                return f"❌ Sorry, I don’t have bus data for {source} → {destination}."

            buses = generate_dummy_buses(source, destination, travel_date)

            response = f"🚌 Available buses from {source} → {destination} on {travel_date}:\n"
            for b in buses:
                response += f"ID {b['id']} - {b['operator']} - {b['time']} - ₹{b['price']}\n"
            response += "\nReply with: `Book ID <bus_id> for <name>`"
            return response
        except:
            return "⚠️ Please follow format: `Ahmedabad to Surat on 2025-09-15`"

    if "book id" in message:
        try:
            parts = message.split("for")
            bus_id = int(parts[0].split("book id")[1].strip())
            name = parts[1].strip().title()

            booking = {"bus_id": bus_id, "name": name}
            bookings.append(booking)
            return f"✅ Ticket booked for {name} on Bus ID {bus_id}.\n(Use `My Bookings` to see all tickets)"
        except:
            return "⚠️ Please follow format: `Book ID <bus_id> for <name>`"

    if "my bookings" in message:
        if not bookings:
            return "📂 No bookings found."
        response = "📂 Your Bookings:\n"
        for b in bookings:
            response += f"ID {b['bus_id']} - Passenger: {b['name']}\n"
        return response

    return "I can help you book buses across Gujarat & India.\nExample: `Ahmedabad to Surat on 2025-09-15`"


In [7]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type your request...")
    clear = gr.Button("Clear")

    def respond(message, chat_history):
        bot_message = bus_booking_agent(message, chat_history)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


/tmp/ipython-input-1017702834.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://023abebe1544ceb08c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
!pip install gradio twilio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.3 MB/s eta 0:00:00
